In [1]:
%run ./FIR_Helper_Methods.ipynb
helper_method = helper_methods()
import traceback

<IPython.core.display.Javascript object>

In [ ]:
class review_ranked_fairness_UI():

    def __init__(self, **kwargs):

        self.ranked_df = kwargs['ranked_df']
        self.protected_attributes = kwargs['protected_attributes']
        self.group_names = kwargs['group_names']
        self.renamed_groups = kwargs['renamed_groups']
        self.encoded_groups = kwargs['encoded_groups']

        self.high_ranking_positive = kwargs['high_ranking_positive']
        self.HIGH_RANGE_POSITIVE = 1
        if self.high_ranking_positive == False:
            self.HIGH_RANGE_POSITIVE = 0

        self.style = {'description_width': 'initial'}
        self.layout = {'width': '700px'}
        self.layout_short = {'width': '300px'}
        self.layout = {'width': 'auto'}
        self.layout_short = {'width': 'auto'}
        
        self.auto_width_layout = {'width': 'auto'}  
        
        self.dictOfSlidersPercent = {}

        self.ranked_df_top = pd.DataFrame()

        # Either upload the file or create the class with pandas Dataframe as input
        self.fileUploader = FileUpload(accept='.csv',
                                       multiple=False,
                                       disabled=False,
                                       style_button='color: darkblue; background-color: lightsalmon; width: 180px;',
                                       compress_level=9
                                       )


        self.selectYAttribute = widgets.Dropdown(
            options=self.ranked_df.columns.values,
            value="Predicted",
            description='Ranked Output:',
            disabled=False,
            style=self.style,
            layout=self.layout
        )

        self.selectImpact = widgets.RadioButtons(
            options={'A Positive impact on the the life of the individual i.e Assistive': 1,
                     'A negative impact on the the life of the individual i.e Punitive': 0},
            description='For decisions, a high rank will results in:',
            disabled=False,
            layout=self.layout,
            style=self.style,
            value=self.HIGH_RANGE_POSITIVE
        )
        
        
        self.view_attributes_button = widgets.Button(
            description='View Full Ranked List',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            layout = self.auto_width_layout
            
        )
        
        self.view_attributes_reduced_list_button = widgets.Button(
            description='View ',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            layout = self.auto_width_layout
            
        )

        self.percentOrTopK = widgets.RadioButtons(
            options={'Top-K results': 1,
                     'Top-K percent of results': 0},
            description='For analysis use:',
            disabled=False,
            layout=self.layout,
            style=self.style,
            value=1
        )

        def on_impact_selected(change):
            if (change['new'] == 1):
                self.HIGH_RANGE_POSITIVE = 1
            if (change['new'] == 0):
                self.HIGH_RANGE_POSITIVE = 0
        self.selectImpact.observe(on_impact_selected, names='value')

        # use to define the selected attributes, one or more
        self.selectProtectedAttributes = widgets.SelectMultiple(
            options=self.ranked_df.columns.values,
            value=self.protected_attributes,
            description='Protected Attribute(s)',
            disabled=False,
            style=self.style,
            layout=self.layout
        )

        # use to select the protected attribute to examine
        self.selectProtectedAttributeToExamine = widgets.Dropdown(
            description='Protected Attributes',
            disabled=False,
            style=self.style,
            layout=self.layout
        )
        
        
         # use to select the protected attribute to examine
        self.selectProtectedAttributeToExamineReferenceGroup = widgets.Dropdown(
            description='Reference Group',
            disabled=False,
            style=self.style,
            layout=self.layout
        )

        self.choosen_attribute_values = widgets.SelectMultiple(
            description='Select attribute value',
            disabled=False,
            style=self.style,
            layout=self.layout_short
        )

        self.selectFairnessMeasure = widgets.Dropdown(
            options=['Counterfactual fairness', 
                     'Equality of outcome',
                     '3'],
            # rows=10,
            
            description='Select the definition of fairness that best applies',
            disabled=False,
            style=self.style,
            layout=self.layout
        )
        
        self.selectFairnessGoal = widgets.Dropdown(
            options={'Equal numbers from each group':1, 
                     'Proportional to representation in population':2,
                     'Proportional to representation in domain':3, 
                     'Proportional to representation in dataset':4,
                     'Individual Fairness':5},
            description='Fairness goal',
            disabled=False,
            style=self.style,
            layout=self.layout
        )
        
        
        
        self.selectWorldView = widgets.Dropdown(
            options={'What you see is what you get':1, 
                     'We are all equal':2},
            description='Fairness worldview',
            disabled=False,
            style=self.style,
            layout=self.layout
        )


        self.selectPercentageOfResults = widgets.Dropdown(
            options=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
            # rows=10,
            description='Top-K percent of results: ',
            disabled=False,
            style=self.style,
            layout=self.layout
        )

        self.selectTopKResults = widgets.IntText(
            value=200,
            description='Top-K results:            ',
            disabled=False,
            style=self.style,
            layout=self.layout,
            max=1000,
            min=0,
        )
        
        
        # Exposure Drop Measure - Relating to the concept of uneven distribution of attention
        self.selectExpDropMeasure = widgets.Dropdown(
            options={'Standard logarithmic discount (using average)':2,
                     'Standard logarithmic discount (using sum)':1,
                     'Geometric Distribution - Biega et al (TBD)':3, 
                     'Viable-n test(TBD)':4},
            # rows=10,
            description='Exposure Drop-off (Position Bias)',
            disabled=False,
            style=self.style,
            layout=self.layout
        )
        
        self.setSignificance = widgets.FloatSlider(
            value=0.1,
            min=0,
            max=1.0,
            step=0.1,
            description='Significance paramater (a):',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
            tooltip = 'A large value means a larger probability of declaring a fair ranking unfair',
            style=self.style,
            layout=self.layout
    )


        self.list_info_out = widgets.Output(layout={})
        self.viewProtectedPandasPlotA = widgets.Output(layout={})
        self.viewFairOutputA = widgets.Output(layout={})
        self.viewReducedListInfoOut = widgets.Output(layout={})
        
        self.upload_data_out = widgets.interactive_output(self.upload_data, {
            'fileUploader_value': self.fileUploader})

        self.select_protected_out = widgets.interactive_output(self.select_protected, {
            'selectProtectedAttributes_values': self.selectProtectedAttributes})

        self.select_ranked_list_col_out = widgets.interactive_output(self.select_ranked_column, {
            'selectYAttribute_values': self.selectYAttribute})
        
        
        self.set_reference_group_options_out = widgets.interactive_output(self.set_reference_group_options, {
            'selectProtectedAttributeToExamine_value': self.selectProtectedAttributeToExamine})

        
        self.viewProtectedOut = widgets.Output(layout={})
        

        
        self.view_analysis_out = widgets.interactive_output(self.run_analysis, {
            'selectExpDropMeasure_value': self.selectExpDropMeasure,
            'setSignificance_value': self.setSignificance})
        

        self.show_hide_ctrls_out = widgets.interactive_output(self.show_hide_ctrls, {
            'percentOrTopK_value': self.percentOrTopK})
        
        self.population_slider_out = widgets.interactive_output(self.create_population_slider,{
             'selectFairnessGoal_value':self.selectFairnessGoal
        })
        

        
        self.worldview_out = widgets.interactive_output(self.worldview_selected, {
            'selectWorldView_value': self.selectWorldView})
        
        

        # to populate the UI at startup - add this to a method later
        self.select_protected(self.selectProtectedAttributes.value)
        self.select_ranked_column(self.selectYAttribute.value)
        
        
        self.view_attributes_button.on_click(self.on_view_attributes_button_clicked)
        with self.view_analysis_out:
            self.run_analysis(self.selectExpDropMeasure.value, self.setSignificance.value)      
    
    #################################################################################################
    #  
    # 
    #################################################################################################
    def upload_data(self, fileUploader_value):
        if not(self.ranked_df.empty):
            self.ranked_df.reset_index(drop=True, inplace=True)
            display("Ranked list already imported")
            display(self.ranked_df.head(5))
        try:
            for key in fileUploader_value:
                display(HTML("uploaded: ",
                      fileUploader_value[key]['metadata']['name']))
                try:
                    csvInBytes = fileUploader_value[key]['content']
                    s = str(csvInBytes, 'utf-8')
                    data = StringIO(s)
                    self.ranked_df = pd.read_csv(data)
                    csvInBytes = 0
                    data = 0
                    s = 0
                    print("")
                    print("Snapshot of unmodified data set")
                    display(self.ranked_df.head(5))
                except:
                    print("Error reading file - upload new file")
        except:
            print("Error uploading file - upload again")
    
    
    
    #################################################################################################
    #  
    # 
    #################################################################################################
    def select_protected(self, selectProtectedAttributes_values):
        try:
            with self.select_protected_out:
                clear_output(wait=True)
                text = "<b><font color='green'> Protected Attributes: </b> </font> " + str(selectProtectedAttributes_values)
                display(HTML(text)) 
                self.selectProtectedAttributeToExamine.options = selectProtectedAttributes_values
                self.selectProtectedAttributeToExamine.value = selectProtectedAttributes_values[0] 
                

        except:
            display("Protected attributes not yet set")

    
    
    #################################################################################################
    #  
    # 
    #################################################################################################
    def select_ranked_column(self, selectYAttribute_values):
        try:
            with self.select_ranked_list_col_out:
                clear_output(wait=True)
                text = "<b><font color='green'> Ranked output column: </b> </font> " + selectYAttribute_values
                display(HTML(text))

        except:
            display("Ranked column not yet set")

   
    

    #################################################################################################
    #  
    # 
    #################################################################################################
    def on_view_attributes_button_clicked (self, b):
        try:
            with self.viewProtectedOut:
                clear_output(wait=True)
                helper_method.view_protected(list(self.selectProtectedAttributes.value),
                                             self.selectYAttribute.value,
                                             self.ranked_df,
                                             _w=600, _h=600,
                                             y_high_positive = self.high_ranking_positive,
                                             persist_impact_col = False,
                                             output_type_binary = False)
                    
        except:
            print("an error occured viewing protected group")
        
      
    
     
    
    #################################################################################################
    #  Show the top-k count or x-percent control depending on the percentOrTopK.value
    # 
    #################################################################################################        
    def show_hide_ctrls(self, percentOrTopK_value):
        if (percentOrTopK_value == 0):
            print ("selectPercentageOfResults", self.selectPercentageOfResults.value)
            self.selectPercentageOfResults.layout.visibility = 'visible'
            self.selectTopKResults.layout.visibility = 'hidden'
        
        elif  (percentOrTopK_value == 1):
            print ("selectPercentageOfResults", self.selectTopKResults.value)
            self.selectPercentageOfResults.layout.visibility = 'hidden'
            self.selectTopKResults.layout.visibility = 'visible'
            
      
    #################################################################################################
    #  Set the contents of the reference group dropdown when the protected feature to examine
    # dropdown (selectProtectedAttributeToExamine) changes value. And recreate the population slider
    # based on the new protectedAttribute and reference group
    ################################################################################################# 
    def set_reference_group_options(self, selectProtectedAttributeToExamine_value):
                print (selectProtectedAttributeToExamine_value)
                self.selectProtectedAttributeToExamineReferenceGroup.options = self.ranked_df[selectProtectedAttributeToExamine_value].unique()
                self.selectProtectedAttributeToExamineReferenceGroup.value = self.selectProtectedAttributeToExamineReferenceGroup.options[0]
                self.create_population_slider(self.selectFairnessGoal.value)   

    #################################################################################################
    #  
    # 
    #################################################################################################   
    def create_population_slider(self, selectFairnessGoal_value):
       
        
        if not self.ranked_df.empty and not self.selectProtectedAttributeToExamine.value == None:
            clear_output(wait=True)
            print ("selectFairnessGoal_value", selectFairnessGoal_value)
            print (self.selectProtectedAttributeToExamine.value)
            pop_in_data = round ((self.ranked_df[self.selectProtectedAttributeToExamine.value]).value_counts(normalize=True)*100, 2)

            
            ''' selectFairnessGoal values
            'Equal numbers from each group':1, 
            'Proportional to representation in population':2,
            'Proportional to representation in domain':3, 
            'Affirmative action for representation in domain':4}'''
            
            desc = ''
            if self.selectProtectedAttributeToExamine.value in self.renamed_groups:
                merged_values = self.renamed_groups[self.selectProtectedAttributeToExamine.value]
                print ("Merged:" ,merged_values)
            if self.selectProtectedAttributeToExamine.value in self.group_names:
                result = self.group_names[self.selectProtectedAttributeToExamine.value]
                print ("Original Group name:" ,result)
            if self.selectProtectedAttributeToExamine.value in self.encoded_groups:
                encoded_values = self.encoded_groups[self.selectProtectedAttributeToExamine.value]
                print ("Encoded:" ,encoded_values)
            try:
                if not self.ranked_df.empty and selectFairnessGoal_value != 1:
                    allGroupsInProtected = self.ranked_df[self.selectProtectedAttributeToExamine.value].unique()

                    for group in allGroupsInProtected:
                        old_name = ""
                        try:
                            old_name = " : " + str(result[str(group)])
                        except:
                            old_name = ""
                        group_col = self.selectProtectedAttributeToExamine.value
                        if selectFairnessGoal_value == 2:
                            desc = group_col + ":" + str(group) + old_name  + ": Percentage of population"
                        
                        elif selectFairnessGoal_value == 3:
                            desc = group_col + ":" + str(group) + old_name + ": Percentage of domain"
                        
                        elif selectFairnessGoal_value == 4:
                            desc = group_col + ":" + str(group) + old_name + ": Percent in dataset"
                        
                        self.dictOfSlidersPercent[group] = widgets.IntSlider(
                                                                                    value=pop_in_data[group],
                                                                                    description= desc ,
                                                                                    max=100,
                                                                                    min=0,
                                                                                    style=self.style,
                                                                                    layout=self.layout)

        
                        display (self.dictOfSlidersPercent[group])
                else: 
                    print ("Equal representation for each group:")
                
            except Exception as e:
                print (e)
                traceback.print_exc()
                print("output not generated yet")  
        
    
    
    
    
    
    #################################################################################################
    #  
    # 
    #################################################################################################        
    def worldview_selected(self, selectWorldView_value):
        
                    
        html_worldview = '''
                    <a href='https://arxiv.org/pdf/1609.07236v1.pdf'>
                    "On the (im)possibility of fairness∗"</a> <br>
                    <a href='https://arxiv.org/pdf/1808.08619.pdf'>
                    "A Comparison of Fairness Definitions under Different Worldviews"</a>'''
           
        
        htmlWSYWYG = HTML('''<br><b>What you see is what you get (WYSIWYG)</b> With respect to the concept of 
                                group fairness this worldview assumes that whatever the data reflects constitutes
                                an objective picture of the world, even if inequalities appear across groups in 
                                the results. In otherwords, that measurable observations objectively reflect 
                                the ability of groups with respect to a goal(succeeding at university, paying back a loan, 
                                not re-offending, excelling in role). For example, if a model predicts that 
                                group A is overall more likely to have a more 
                                favorable distribution of positive outcomes than Group B a WYSIWIG worldview 
                                would lead one to conclude, well, that's just the way it is.<br><br>''')



        htmlWAE = HTML('''<br><b>We are all equal (WAE)</b> With respect to the concept of group fairness this 
                            worldview assumes that data does not always constitute an objective picture of the
                            world, as structural or systemic bias is inevitibely contained within it, therefor
                            when the distribution of measurable observations differ accross groups this should not be 
                            mistaken for a difference in *distribution of ability* with respect to a goal(succeeding at 
                            university, paying back a loan, not re-offending, excelling in role) but rather this may indicate
                            a fundamental difference in conditions across groups that has resulted in the variance in 
                            distribution of measurable observations. As a result we might apply a mathematical 
                            compensation for any major discrepencies. This worldview may example result in the 
                            observation that as there is no biological basis for race, a difference
                            in the distribution of positive outcomes across race 1 compared to race 2 can 
                            only be accounted for by external factors outside of the control of the individuals constituting
                            race 1 and as such should be mathematically addressed.<br><br>''')
        
        if (selectWorldView_value == 1):
            
            display( htmlWSYWYG)
        
        elif  (selectWorldView_value== 2):
            display( htmlWAE)
            
            
    #################################################################################################
    #  
    # 
    ################################################################################################# 
    def plot_relevance_exposure_ratio(self, df, protected_feature, reference_group, _count):
        display (df)
        '''Standard logarithmic discount (using average)':2,
         'Standard logarithmic discount (using sum)':1,
         'Geometric Distribution - Biega et al (TBD)':3, 
         'Viable-n test(TBD)':4}'''
        measure = self.selectExpDropMeasure.value
        if measure == 1:
             text = '''Detect Inequality of attention, using a standard logarithmic discount to calculate
             as discussed in
             
             '''
        elif measure == 2:
            text = '''Detect Inequality of attention, using a standard logarithmic discount 
            to measure exposure drop off and hence exposure for each position in the ranked list 
            as discussed by <b>Joachims and Singh</b> in "Fairness of Exposure in Rankings"
            
            comparing the sum of relevance per group against the sum of exposure per group 
             
             '''
            
        elif measure == 3:
            
        elif measure == 4:
        
        # convert Series to Python strings
        #fig.update_layout(barmode='group')
        _x = []
        _y = []
        for val in df[protected_feature]:
            _x.append(val)
            _y.append (_count[val])
            
        fig = px.bar(df, 
                     x=protected_feature, 
                     y='exposure',
                     text=round(df['exposure'],2),
                     barmode='group',
                     color_discrete_sequence =['lightsalmon']*len(df),
                     height=400)
        fig.update_traces(texttemplate='%{text}', textposition='outside')
        fig.update_layout(xaxis_type='category',
                          title_text='Exposure')
        fig.update_xaxes(tickangle=45, 
                         showline=True, 
                         linewidth=2, 
                         linecolor='black',
                         tickfont=dict(family='Rockwell', 
                                       color='green', 
                                       size=16))
        fig.update_yaxes(showline=True, 
                         linewidth=2, 
                         linecolor='black', 
                         range=[0, df['exposure'].max()+ (df['exposure'].mean()/2)])
        fig_1 = go.FigureWidget(fig)
        del fig
                         
        #######                 
        fig = px.bar(df, 
                     x=protected_feature, 
                     y='relevance', 
                     text=round(df['relevance'],2),
                     barmode='group',
                     color_discrete_sequence =['indianred']*len(df),
                     height=400)
        fig.update_traces( texttemplate='%{text}', textposition='outside')
        fig.update_layout(xaxis_type='category',
                          title_text='Relevance')
        fig.update_xaxes(tickangle=45, 
                         showline=True, 
                         linewidth=2, 
                         linecolor='black',
                         tickfont=dict(family='Rockwell', 
                                       color='green', 
                                       size=16))
        
        fig.update_yaxes(showline=True, 
                         linewidth=2, 
                         linecolor='black', 
                         range=[0, df['relevance'].max() + (df['relevance'].mean()/2)])
        fig_2 = go.FigureWidget(fig)
        del fig
        #############
        fig = px.bar(x=_x, 
                     y=_y,
                     text=_y,
                     barmode='group',
                     color_discrete_sequence =['lightsalmon']*len(df),
                     height=400,
                     labels = {'x':protected_feature,
                               'y': 'Count'
                              }
                    )
        fig.update_traces(texttemplate='%{text}', textposition='outside')
        fig.update_layout(xaxis_type='category',
                          title_text='Count')
        
        fig.update_xaxes(tickangle=45, 
                         showline=True, 
                         linewidth=2, 
                         linecolor='black',
                         tickfont=dict(family='Rockwell', 
                                       color='green', 
                                       size=16))
        fig.update_yaxes(showline=True, 
                         linewidth=2, 
                         linecolor='black', 
                         range=[0, max(_y)+25])
        fig_3 = go.FigureWidget(fig)
        del fig
        
        
        display(widgets.HBox([fig_1, fig_2, fig_3]))
        
    
    def get_length_list_to_analyze (self, df):
        if not self.ranked_df.empty:
            # Now reduce the ranked list based on whethere we are using percentage or top k
            if self.percentOrTopK.value == 0:
                cut_results_len=round(len(df) *selectPercentageOfResults_value/100)
            elif self.percentOrTopK.value == 1:
                cut_results_len= self.selectTopKResults.value
            try:
                with self.list_info_out:
                    clear_output(wait=True)
                    text_a = 'Total number of records in ranked list (n) = <b>' + str(len(df)) + '</b>'
                    text_b = '<br>Number of unique values in ranked output = <b>' + str(len(df[self.selectYAttribute.value].unique())) + '</b>'
                    text_c ='''<br>Note: when n is sufficiently small we expect the user to read all results and 
                    therefor for all outcomes to be equal.'''
                    text_d = '''
                    <b>The probability ranking principle:</b> states that the ideal ranking should order items in
                    the decreasing order of their probability of relevance(as this maximises <b>apparent</b> utility or 
                    usefulness).
                    '''
                    helper_method.display_html(text_d, 'black', "p")
                    text = text_a + text_b + text_c
                    helper_method.display_html(text, helper_method.text_color, "p")
 
            except:
                display ('Data not yet loaded')
            return cut_results_len
        
    
    #################################################################################################
    #  
    # 
    #################################################################################################
   
    def run_analysis(self, selectExpDropMeasure_value, setSignificance_value):
        
        if not self.ranked_df.empty and not self.selectProtectedAttributeToExamine.value == None:
            local_ranked_df = self.ranked_df.copy()
            cut_results_len = self.get_length_list_to_analyze(self.ranked_df)
            self.ranked_df_top=self.ranked_df.head(cut_results_len).copy()
            ##local interact method - Examine the new reduced ranked list
            def show_view_protected (button_status):
                if button_status == True:
                    helper_method.view_protected(list(self.selectProtectedAttributes.value),
                                                      self.selectYAttribute.value,
                                                      self.ranked_df_top, 
                                                      _w=600, _h=600,
                                                      y_high_positive = self.high_ranking_positive,
                                                      persist_impact_col = False,
                                                      output_type_binary = False,
                                                      show_outcome_info = False)
            with self.viewReducedListInfoOut:
                clear_output(wait=True)
                interact(show_view_protected, button_status = widgets.ToggleButton(
                                                description='View Reduced Ranked List',
                                                disabled=False)
                                                )

        
            #local interact method
            def calc_exposure_drop_off(selectExpDropMeasure_value):
                expList = []
                if (selectExpDropMeasure_value == 1):
                    #Calculate the Logrithmic discount exposure drop off for the entire dataset 
                    #and use the average in the calculations        
                    for x in range(len(local_ranked_df)):
                    # For each record add the exposure to a pandas Series
                        exposure=1/math.log(x+2, 2)
                        expList.append(exposure)
                
                elif (selectExpDropMeasure_value == 2):
                    #Calculate the Logrithmic discount exposure drop off for the entire dataset 
                    #and use the sum in the calculations 
                    for x in range(len(local_ranked_df)):
                        #For each record add the exposure to a pandas Series
                        exposure=1/math.log(x+2, 2)
                        expList.append(exposure)
                
                elif (selectExpDropMeasure_value == 3):
                    #Calculate the geometric distribution exposure drop off for the entire dataset
                    for x in range(len(local_ranked_df)):
                        #For each record add the exposure to a pandas Series
                        exposure=1/math.log(x+2, 2)
                        expList.append(exposure)
        
                elif (selectExpDropMeasure_value == 4):
                    #Calculate the Logrithmic discount exposure drop off for the entire dataset
                    for x in range(len(local_ranked_df)):
                        #For each record add the exposure to a pandas Series
                        exposure=1/math.log(x+2, 2)
                        expList.append(exposure)
                
                else:
                    for x in range(len(local_ranked_df)):
                    #For each record add the exposure to a pandas Series
                        exposure=1/math.log(x+2, 2)
                        expList.append(exposure)
                       
                # Now we have a full list with the "logarithmic discount" exposure
                local_ranked_df.insert(0, "exposure", expList, True)
                self.ranked_df_top=local_ranked_df.head(cut_results_len).copy()
                
                #now we have a local copy of the full ranked list and the reduced ranked list with exposure 
                if not self.ranked_df_top.empty:
                    _count = self.ranked_df_top.groupby(self.selectProtectedAttributeToExamine.value).count()['Predicted']
                    adjustment = 0
                    lowest_utility = self.ranked_df[self.selectYAttribute.value].min()
                    #Adjust all relevance scores if any are less than 0
                    if (lowest_utility < 0):
                        display(HTML("<font color='blue'><b>Note: </b>Relevance score adjusted by min value " + str(round(lowest_utility,3)) + " to ensure no negative numbers in measure</font>") )
                        adjustment = lowest_utility
                        local_ranked_df['Predicted'] = local_ranked_df['Predicted'].apply(lambda x: x-adjustment)
                    #######

                    if (selectExpDropMeasure_value == 1):
                        #Create a small data frame containing the sum of exposure and relevance per group
                        full_exp_util_df=local_ranked_df.groupby(self.selectProtectedAttributeToExamine.value).sum()[['exposure','Predicted']]
                        reduced_exp_util_df=self.ranked_df_top.groupby(self.selectProtectedAttributeToExamine.value).sum()[['exposure','Predicted']]
        
                
                    elif (selectExpDropMeasure_value == 2):
                        #Create a small data frame containing the average of exposure and relevance per group
                        full_exp_util_df=local_ranked_df.groupby(self.selectProtectedAttributeToExamine.value).mean()[['exposure','Predicted']]
                        reduced_exp_util_df=self.ranked_df_top.groupby(self.selectProtectedAttributeToExamine.value).mean()[['exposure','Predicted']]
   
                    #else  #3, 4, 5 to be implemented 
                     
                    full_exp_util_df['exposure_over_relevance']=full_exp_util_df.apply(lambda row: row.exposure/row.Predicted, axis=1)
        
                    full_exp_util_df.rename(columns={'Predicted':'relevance'}, inplace=True)
                        
                    display(HTML("List reduced to top: " + str (cut_results_len)))
                        
                    # Create a ranked list of only the "top" records.
                    reduced_exp_util_df['exposure_over_relevance']=reduced_exp_util_df.apply(lambda row: row.exposure/row.Predicted, axis=1)
                    reduced_exp_util_df.rename(columns={'Predicted':'relevance'}, inplace=True)
                    self.plot_relevance_exposure_ratio(reduced_exp_util_df.reset_index(), 
                                                       self.selectProtectedAttributeToExamine.value,
                                                       self.selectProtectedAttributeToExamineReferenceGroup.value,
                                                        _count)
                        
                    display (HTML("Full dataset:"))
                    self.plot_relevance_exposure_ratio(full_exp_util_df.reset_index(), 
                                                           self.selectProtectedAttributeToExamine.value,
                                                          self.selectProtectedAttributeToExamineReferenceGroup.value,
                                                          _count)
            
            
            interact(calc_exposure_drop_off, selectExpDropMeasure_value = self.selectExpDropMeasure)
            #just clear the view_analysis_output as we are about to use it again
            #Populate the input to the FA*IR Analysis
#             with self.viewFairOutputA:
#                 clear_output(wait=True)
#                 display(HTML("Larger values for significance(a) indicate a stricter necessity to adhere to the required porportion of protected elements at each position"))
                
#                 display(self.group_names[selectProtectedAttributeToExamine_value])
#                 allGroupsInProtected = self.ranked_df[selectProtectedAttributeToExamine_value].unique()
#                 for group in allGroupsInProtected:
#                     display(HTML("Required percentage for group "+ str(group) +" is " + str (self.dictOfSlidersAffermativeActionGoal[group].value)))
                
#                 display(HTML("K =: " + str (cut_results_len)))
#                 for group in allGroupsInProtected:
#                     display(HTML("Targeted minimum proportion(p) for group "+ str(group) +" is " + str (self.dictOfSlidersAffermativeActionGoal[group].value/100)))
                
#                 display(HTML("a =: " + str (self.setSignificance.value)))
            
        
    def render(self):
        space=widgets.Label('  ', layout=widgets.Layout(width='100%'))

        uploadFileHTML=widgets.HTML(
            "<h3><left>1. Upload Ranked list:</left></h3>")

        setRankedOutputHTML=widgets.HTML(
            "<h3><left>2. Identify the ranked output column:</left></h3>")

        effectOfRankingHTML=widgets.HTML(
            "<h4><left>Set the effect of ranking on the individual or group</left></h4>")

        setProtectedAttributesHTML=widgets.HTML(
            "<h3><left>3. Identify the protected attributes column:</left></h3>")

        visualiseProtectedAttributesHTML=widgets.HTML(
            "<h3><left>4. Visualise protected attributes based on full ranked list:</left></h3>")

        analyzeProtectedHTML=widgets.HTML(
            "<h3><left>5. Analyze ranking for fairness:</left></h3>")

        analyzeProtectedHTML_B=widgets.HTML(
            "<h4><left>Specify the number of results that will receive attention:</left></h4>")

        analyzeProtectedHTML_C=widgets.HTML(
            "<h4><left>Specify the protected attribute to analyse:</left></h4>")
        
        analyzeProtectedHTML_C_2=widgets.HTML(
            "<br><h4><left>Specify the protected group to use as a reference wrt Fairness:</left></h4>")
        
        analyzeProtectedHTML_D=widgets.HTML(
            "<h4><left>Specify the worldview that will be supported by the ranking model:</left></h4>")
        
        analyzeProtectedHTML_E=widgets.HTML(
            "<h4><left>Specify the fairness goal to be supported by the ranking model:</left></h4>")
        
        
        
        resultsHTML=widgets.HTML("<h3><left>Results:</left></h3>")
       
    
    
        JoacmAndSinghHTML=widgets.HTML(
            '''<h4><left>Joachims and Singh Fairness of Exposure in Ranking (Balancing fairness and perceived utility):</left></h4><h5>
            Is exposure fairly allocate(regardless of how relevance/utility is estimated)</h5>
            
            <a href="https://arxiv.org/pdf/1802.07281.pdf">"Fairness of Exposure in Rankings"</a>

         ''')
        
        exposureHTML = widgets.HTML('''
        <html><p style="display:inline; line-height:0px"><b>Exposure:</b> There is little doubt that the order of ranking is correlated with the 
                    result which receive the majority of the user attention and interaction and that our 
                    cognitive inclinations tend to place a higher value on results positioned at the top of a 
                    list. The results of a 2008 study (by M. Keane, M. O'Brien, & B. Smyth) has shown definite 
                    evidence of bias in a user’s interaction with a list returned by a search, whereby the user 
                    tends to give preferences to items at the top of the list, although they do sometimes seek 
                    out relevant results with a lower ranking. The measure of this attention is known as 
                    "<b>exposure</b>" and the exposure a person or item will receive is largely determined by the position 
                    in which they will appear in the final ranking displayed to the end user. 

                    <br><br>We should consider if the absolute maximization of <b>apparent</b> utility (relevance) is
                    appropriate or fair when ranking humans or if we may want to impose some fairness constraints 
                    that might guarantee a particular notion of fairness. Due to the nature of ranking a 
                    small difference in utility(relevance) may result in a large difference in exposure</p><br>
                    
            <b>Note:</b> Using standard log discount with average is not useful when there is very low 
            representation in one group. 
            
            Utility maximation subject to fairness.
        ''')

    
        FAIRHTML=widgets.HTML(
            "<h4><left>FA*IR: A Fair Top-k Ranking</left></h4><h5>Ranked group fairness: Does the representation of the protected group fall below a minimum proportion(p) at any point in the ranking (regardless of how relevance is estimated)</h5>")
         
        
        display(
            uploadFileHTML,
            self.fileUploader,
            self.upload_data_out,
            
            
            space,
            setRankedOutputHTML,
            self.selectYAttribute,
            self.select_ranked_list_col_out,
            
            
            space,
            effectOfRankingHTML,
            self.selectImpact,
            
            
            space,
            setProtectedAttributesHTML,
            self.selectProtectedAttributes,
            self.select_protected_out,
            
            
            space,
            visualiseProtectedAttributesHTML,
            self.view_attributes_button,
            self.viewProtectedOut,

            space,
            analyzeProtectedHTML,
            # view_analysis input fields
           
            self.list_info_out,
            space,
                
            analyzeProtectedHTML_B,
            self.percentOrTopK,
            self.selectTopKResults,
            self.selectPercentageOfResults,
            self.viewReducedListInfoOut,
            space,
                
            analyzeProtectedHTML_C,
            self.selectProtectedAttributeToExamine,
            analyzeProtectedHTML_C_2,
            self.selectProtectedAttributeToExamineReferenceGroup,
            self.set_reference_group_options_out,
                
            space,
            analyzeProtectedHTML_D,
            self.selectWorldView,
            self.worldview_out,
            analyzeProtectedHTML_E,
            self.selectFairnessGoal,
            self.population_slider_out,
            space,

            space,
            resultsHTML,
            exposureHTML,
            JoacmAndSinghHTML,
            space,

            self.view_analysis_out,
            self.viewProtectedPandasPlotA,
            space,
            FAIRHTML,
            space,
            self.setSignificance,
            self.viewFairOutputA,
            space,
            )

In [ ]:
'''computer scientists typically associate fairness with a property of unbounded nondeterminism while 
economists believe fairness is an attribute of the rational valuation of goods. 
Culturally, fairness is associated with behaviors that are not influence by person’s skin color, 
race or socio-economic status. As per the proposal in we can consider fairness through two world views.
If we look at the distribution of an outcome across two groups, group 1 and group 2. 
Individual vs. Group: Another friction that defines fairness in machine learning algorithms is
Group vs. Individual. Group fairness, in its broadest sense, partitions a population into groups
defined by protected attributes and seeks for some statistical measure to be equal across groups. 
Individual fairness, in its broadest sense, seeks for similar individuals to be treated similarly.



Individual fairness could be considered as groups of size 1
'''

demographicParityHTML = '''Demographic Parity states that the proportion of each group within a protected feature/attribute
                            (e.g. gender, race) should receive a positive outcome at equal rates. 
                            Where a positive outcome is the outcome that will benefit the human eg. 
                            “Getting to university”, 
                            “Getting a loan”, 
                            “Not getting flagged for potential recedivism”,
                            "Getting interviewed for the job",
                            "Not getting flagged for potential fraud",
                            "Not getting flagged as potential criminal"
                             Ideally the proportions will be equal. A motivation for demographic parity will be to 
                             change the state of our current world to improve it 
                             (e.g.: we want to see more minority groups getting to the top).
                             Awareness that historical biases may affected the quality of our data 
                             (e.g.: ML solution trained to hire software engineers, where nearly no women was 
                             hired before)
                            Ideally thre is a plan in place to support the unprivileged group and to prevent the
                            reinforcement of historical biases 
                            (e.g.: setting policies that penalise non inclusive behaviour at a board)'''

    
